In [ ]:
# Setup and imports
import re
import os
from pathlib import Path

import numpy as np
import mne
# from IPython.display import clear_output

from src.MovieEEGSourcePipeline.source import _load_epochs

os.chdir("/Users/yeganeh/Codes/MovieEEG-SourcePipeline/")

In [17]:
# Config
# -----------------------------
SFREQ = 512
DATA_DIR = Path("data/epochs")
SUBJECTS_DIR = Path("data")          # contains fsaverage/ (i.e., data/fsaverage)
FS_SUBJECT = "fsaverage"             # we use fsaverage anatomy for everyone

# Use an ico-4 source space (coarse; appropriate for Yeo-7 parcellation)
FS_SRC_FNAME = SUBJECTS_DIR / FS_SUBJECT / "bem" / "fsaverage-ico-4-src.fif"
FS_BEM_FNAME = SUBJECTS_DIR / FS_SUBJECT / "bem" / "fsaverage-5120-5120-5120-bem-sol.fif"

MONTAGE_NAME = "standard_1020"
os.environ["SUBJECTS_DIR"] = str(SUBJECTS_DIR)  # a temporary solution to make sure mne can find SUBJECTS_DIR

In [ ]:
# Atlas labels (fsaverage annotations)
atlas_labels = mne.read_labels_from_annot(
    subject=FS_SUBJECT,
    parc="Yeo2011_7Networks_N1000",
    subjects_dir=str(SUBJECTS_DIR),
    verbose=False,
)

# Pick a single example file to define channel set for forward model
example = next(DATA_DIR.glob("*_art_nl_epo.fif"))
fwd = make_forward(example)

inv_cache = {}  # subject -> inverse operator built from baseline1

for epochs_path in sorted(DATA_DIR.glob("*_epo.fif")):
    m = re.search(r"^(\d+)_([^_]+_[^_]+)_epo$", epochs_path.stem)
    if m is None:
        continue
    subject, film = m.groups()
    epochs = _load_epochs(epochs_path)

    out_dir = Path("data/labels")
    out_dir.mkdir(parents=True, exist_ok=True)
    output_path_labels = out_dir / f"{subject}-{film}-labels.npz"

    if output_path_labels.exists():
        continue

    # Ensure we have an inverse per subject (from baseline1)
    if subject not in inv_cache:
        # pick baseline portion (avoid immediate pre-cut if you worry about anticipatory activity)
        epochs_base = epochs.copy().crop(tmin=-0.2, tmax=-0.05)
        inv_cache[subject] = make_inverse_from_baseline(epochs_base, fwd)

    inv = inv_cache[subject]

    print(f">>>>>>>> {subject} {film}")
    label_ts = extract_label_time_series(epochs, inv, atlas_labels)
    np.savez_compressed(output_path_labels, labels=label_ts)